In [ ]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


**Setup**

Si la instalación de lightgbm da error, ejecutar en Anaconda Prompt 

`conda install -c conda-forge xgboost=1.4.0`

`conda install -c conda-forge lightgbm=3.2.1`

---

<img src='../../../common/logo_DH.png' align='left' width=35%/>


# Boosting

<a id="section_introduccion"></a> 
## Introducción
Como mencionamos en la parte I, existen modelos de árboles de decisión simples, y árboles de decisión de Ensamble. Vimos también que los primeros, si bien son fáciles de interpretar, no pertenecen al grupo de modelos que ofrecen mayor precisión y que ésto se debía en gran medida a la varianza propia de dichos modelos. Como alternativa superadora surgieron los árboles de decisión por ensamble. Los modelos de Ensamble logran reducir la varianza inherente de los árboles de decisión y han logrados muy buenos resultados en cuanto a la precisión alcanzada, sin embargo ésto es a costa de perder la facilidad de interpretación que poseían los árboles de decisión simples.  
  

<a id="section_boosting"></a> 
### Boosting:
Boosting es la tercera técnica de agregación que veremos. La principal diferencia con las técnicas que vimos anteriormente, es que en ellas se entrenaban los modelos independientemente para luego generar un meta-modelo. En el caso de Boosting, se entrenan los modelos de manera secuencial donde cada modelo aprende de los errores del modelo predecesor. 

Las técnicas de boosting más conocidas son:
- ADA Boost
- Gradient Boosting
- XG Boost

Veamos cada una de ellas:



**1) ADA Boost:**

El nombre ADA proviene de Adaptative Boosting, que hace referencia a su capacidad de Adaptar la importancia de los predictores asignándole mayor peso a aquellos sobre los que se comete más error. Es importante destacar que mientras en Random Forest mencionamos que a los árboles no se los poda, en el casode Adaboost sucede todo lo contrario: se suelen usar árboles de 1 nodo raíz y 2 nodos hojas. A este tipo de árboles se los conoce como **stump**. Por otro lado, mientras que en Random Forest cada árbol tenía igual voto sobre la predicción final, en el caso de ADA Boost tenemos que los votos de los **stumps** pueden tener más pesos unos que otros.  

Tal como mencionamos los métodos de boosting trabajan por definición secuencialmente, con lo cual cada **stump** va a "aprender" de la secuencia anterior. Al primer stump se lo va a entrenar con un dataset al cual se le va a asignar los mismos pesos a cada observación (fila). De esta manera si nuestro dataset tiene un total de K observaciones, entonces cada observación tendrá un peso de 1/K siendo todas igual de "importantes". 

Luego se elige el feature que genera la menor entropía o gini y se crea el primer stump con dicho feature. Calculamos ahora la importancia de dicho árbol (o cuanto peso tendrá su voto sobre la predicción del meta-modelo) dependiendo de la cantidad de error que cometió. Este error lo calculamos sumando los pesos de todas las observaciones que fueron mal clasificadas (este valor va a estar dentro del rango entre 0 y 1). 
- Si es 1 significa que no logró clasificar nada correctamente y por ende tendra muy poco voto en la predicción final.
- Si es 0 significa que clasifico todo perfectamente y por ende tendrá mayor voto en la predicción final.

A su vez, para asegurarnos que el siguiente stump pueda aprender del precursor, realizamos un ajuste de los pesos de cada observación del dataset. Ahora ya no serán todas las observaciones de igual peso o importancia, sino que se le dará mayor peso a aquellos observaciones que fueron mal clasificadas, y consecuentemente se le restará peso a aquellas que fueron correctamente clasificadas, de modo que siempre la suma total de pesos sea igual a 1.

Ahora el segundo Stump va a utilizar weighted Gini index para seleccionar la mejor partición, y a continuación se repiten todos los pasos:
- se calcula el error total de este stump para asignarle el peso a su voto.
- se vuelven a recalcular los pesos de cada observacion para entrenar al siguiente stump.


<img style="float: center;" src="img/Adaboost_01.png"> 

En este [link](https://www.youtube.com/watch?v=k4G2VCuOMMg) podemos ver el algoritmo en acción.

Este algoritmo es muy potente pero tiene desventajas como:

* Puede producir overfitting (el peso a los outliers va creciendo)
* No es interpretable
* No es multiclase (existen variantes como Adaboost.M1 que sí lo son)


La predicción del meta-modelo estará luego conformada de la siguiente manera:
- Clasificador: 
    - Voto con pesos
    - En sklearn: `AdaBoostClassifier`
- Regresión: 
    - Promedio ponderado
    - En sklearn: `AdaBoostRegressor`








**2) Gradient Boost:**

Gradient boosting es un método de aprendizaje lento donde los sucesivos modelos de árboles de decisión son entrenados para predecir los residuales del árbol antecesor permitiendo que los resultados de los modelos subsiguientes sean agregados y corrijan los errores promediando las predicciones. Para determinar los parámetros que tendrán cada uno de los árboles de decisión agregados al modelo se utiliza un procedimiento descenso por gradiente que minimizará la función de perdida. De esta forma se van agregando árboles con distintos parámetros de forma tal que la combinación de ellos minimiza la pérdida del modelo y mejora la predicción. 

La diferencia con adaboost es que ya no pesamos cada punto independientemente, sino que proponemos una función de error cuyo gradiente tenemos que minimizar. El hiperparámetro de Learning Rate ($\eta$) es un escalar entre 0 < $\eta$ < 1 que multiplica los residuales para asegurar convergencia. A medida que se reduce el valor de $\eta$ es recomendable aumentar el número de estimadores N.

<img style="float: center;" src="img/Gradientboost_01.png"> 

La predicción del meta-modelo estará luego conformada de la siguiente manera:

$y_{pred} = y_1 + \eta r_1 + ... +  \eta r_N$


Árboles de decisión con Gradient boosting es uno de los modelos más poderosos y más utilizados para problemas de aprendizaje supervisado. Su principal inconveniente es que requieren un ajuste cuidadoso de los parámetros y puede requerir mucho tiempo de entrenamiento.

En sklearn:  
- Regresión: `GradientBoostingRegressor`
- Clasificador: `GradientBoostingClassifier`

La implementación den Scikit-Learn toma los siguientes parámetros:

- `base_estimator`: el estimador sobre el cual se va a construir el ensamble. Por efecto, son árboles de decisión.
- `n_estimators`: cantidad de estimadores que se van a utilizar
- `learning_rate`: un numero bajo de Learning rate asegura convergencia del descenso del gradiente, pero aumenta los tiempos de entrenamiento. Como mencionamos anteriormente, a medida que se reduce el valor de $\eta$ es recomendable aumentar el número de estimadores N.

**3) XG Boost:**

XGBoost significa e**X**treme **G**radient Boosting. Es el algoritmo que ha estado dominando recientemente los problemas Machine learning y las competiciones de Kaggle con datos estructurados. Es un caso especifico de gradient boosting donde se aplica Regularizacion a la funcion de costo lo cual ayuda a evitar overfitting.  

Internamente, XGBoost representa todos los problemas como un caso de modelado predictivo de regresión que sólo toma valores numéricos como entrada. Si nuestros datos están en un formato diferente, primero vamos a tener que transformarlos para poder hacer uso de todo el poder de esta librería. El hecho de trabajar sólo con datos numéricos es lo que hace que esta librería sea tan eficiente.

Para utilizar XGBoost deben importar la libreria: `import xgboost as xgb`
- Regresión: `xgb.XGBRegressor()`
- Clasificador: `xgb.XGBClassifier()`

Los argumentos que toma como input son ya conocidos:

- `learning_rate`: un numero bajo de Learning rate asegura convergencia del descenso del gradiente, pero aumenta los tiempos de entrenamiento. Como mencionamos anteriormente, a medida que se reduce el valor de $\eta$ es recomendable aumentar el número de estimadores N.
- `n_estimators`: cantidad de estimadores que se van a utilizar
- `criterion`: define el criterio de impureza para evaluar la calidad de las particiones
- `max_features`: la cantidad de features que extraerá para entrenar cada `base_estimator`. Por default es igual a `sqrt(X.shape[1])`
- `bootstrap` y `bootstrap_features`: controla si tanto los n_samples como las features son extraidos con reposición.
- `max_depth`: la pronfundidad máxima del árbol
- `min_samples_leaf`: el número mínimo de n_samples para constituir una hoja del árbol (nodo terminal)
- `min_samples_split`: el número mínimo de n_samples para realizar un split.



## Medición de Performance en modelos de clasificación:
A continuación aconsejamos ver el siguiente video que explica las Curvas ROC y el AUC como mediciones de performance de los modelos de clasificación.

In [ ]:
from IPython.display import HTML, IFrame

# Youtube
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/AcbbkCL0dlo" frameborder="0" allowfullscreen></iframe>')

---

<a id="section_referencias_2"></a> 
## Referencias 

[volver a TOC](#section_toc_2)


https://en.wikipedia.org/wiki/Gradient_boosting

https://xgboost.readthedocs.io/en/latest/index.html

https://relopezbriega.github.io/blog/2017/06/10/boosting-en-machine-learning-con-python/

https://www.kaggle.com/stuarthallows/using-xgboost-with-scikit-learn   